In [ ]:
from keras.datasets import mnist
import numpy as np

def balanced_mnist_set():
  (x_train, y_train), (x_test, y_test) = mnist.load_data()

  # Se obtiene una muestra con igual número de: cincos vs no cincos
  # para el conjunto de entrenamiento y el de test
  idx_y_train_5 = np.where(y_train == 5)[0]
  idx_y_train_n5 = np.where(y_train != 5)[0]
  idx_y_train_n5 = np.random.choice(idx_y_train_n5, idx_y_train_5.shape[0], replace=False)

  idx_y_train = np.concatenate([idx_y_train_5, idx_y_train_n5])
  np.random.shuffle(idx_y_train)

  x_train = x_train[idx_y_train]
  y_train = y_train[idx_y_train]
  #print(x_train.shape)

  idx_y_test_5 = np.where(y_test == 5)[0]
  idx_y_test_n5 = np.where(y_test != 5)[0]
  idx_y_test_n5 = np.random.choice(idx_y_test_n5, idx_y_test_5.shape[0], replace=False)

  idx_y_test = np.concatenate([idx_y_test_5, idx_y_test_n5])
  np.random.shuffle(idx_y_test)

  x_test = x_test[idx_y_test]
  y_test = y_test[idx_y_test]
  #print(x_test.shape)

  y_train_binary = (y_train == 5).astype(int)
  y_test_binary = (y_test == 5).astype(int)
  return (x_train, y_train_binary), (x_test, y_test_binary)

In [ ]:
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Flatten, Dense
from sklearn.metrics import confusion_matrix, \
precision_score, recall_score,f1_score

(x_train, y_train_binary), (x_test, y_test_binary) = balanced_mnist_set()
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(1)
])

model.compile(loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train_binary, epochs=4, verbose=0,
                    batch_size=2000, validation_split=0.9)

loss_train = history.history['loss']
loss_val = history.history['val_loss']
accuracy_train = history.history['accuracy']
accuracy_val = history.history['val_accuracy']
epochs = range(1, len(loss_train) + 1)

# Loss
plt.figure(figsize=(12, 6))
plt.plot(epochs, loss_train, label='loss')
plt.plot(epochs, loss_val, label='val_loss')
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.grid(True)
plt.show()

# Accuracy
plt.figure(figsize=(12, 6))
plt.plot(epochs, accuracy_train, label='accuracy')
plt.plot(epochs, accuracy_val, label='val_accuracy')
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.grid(True)
plt.show()


y_pred = (model.predict(x_test, verbose=0) >= 0.5).astype(int)[:,0]

print(precision_score(y_test_binary, y_pred),
      recall_score(y_test_binary, y_pred),
      f1_score(y_test_binary, y_pred),
      precision_score(y_test_binary, y_pred, pos_label=0),
      recall_score(y_test_binary, y_pred, pos_label=0),
      f1_score(y_test_binary, y_pred, pos_label=0))